In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import timm

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
#from pyts.image import GramianAngularField, MarkovTransitionField
from io import BytesIO
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

2025-04-06 15:36:20.337206: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 15:36:20.355594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-06 15:36:20.376095: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-06 15:36:20.382475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 15:36:20.397874: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("TF version",tf.__version__)
print("NUm GPUs available:",len(tf.config.list_physical_devices("GPU")))
# print(tf.config.list_physical_devices)
# print(tf.config.get_visible_devices_devices())
# print("is_built_with_cuda:", tf.test.is_built_with_cuda())
# print("is_built_with_cuda:", tf.test.is_built_with_gpu_support())

TF version 2.17.0
NUm GPUs available: 0


2025-04-06 15:36:26.966007: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# For shuffle
SEED = 42

In [26]:
BASE = '/home/HardDisk/Satang/thesis_proj'

In [27]:
DATAPATH = 'original'

In [28]:
FOLDER = "win7-120gb-hdd"

In [29]:
os.chdir(f'{BASE}/{DATAPATH}')
folders = sorted(os.listdir())
print(folders)

['win7-120gb-hdd', 'win7-120gb-ssd', 'win7-250gb-hdd', 'win7-250gb-ssd']


In [30]:
os.chdir(f'{BASE}/{DATAPATH}/{FOLDER}')
labels = sorted(os.listdir())
print(labels)

['AESCrypt', 'Cerber', 'Darkside', 'Excel', 'Firefox', 'GandCrab4', 'Ryuk', 'SDelete', 'Sodinokibi', 'TeslaCrypt', 'WannaCry', 'Zip']


In [31]:
benign = ['AESCrypt', 'Zip', 'SDelete', 'Excel', 'Firefox']
ransomware = ['TeslaCrypt', 'Cerber', 'WannaCry', 'GandCrab4', 'Ryuk', 'Sodinokibi', 'Darkside']

In [ ]:
import os
import shutil

def delete_folders(source_dir, folders_to_delete):

    folders_deleted = 0

    for folder_name in folders_to_delete:
        folder_path = os.path.join(source_dir, folder_name)

        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            shutil.rmtree(folder_path)  # Deletes the folder and its contents
            folders_deleted += 1
            print(f"Deleted: {folder_name}")
        else:
            print(f"Folder not found: {folder_name}")

    print(f"Total folders deleted: {folders_deleted}")

# Example usage:
source_directory = "/home/HardDisk/Satang/thesis_proj/original/win7-120gb-ssd"
folders_list = [
    "Cerber-largefiles", "Cerber-w10dirs",
    "Darkside-largefiles", "Darkside-w10dirs",
    "GandCrab4-largefiles", "GandCrab4-w10dirs",
    "Ryuk-largefiles", "Ryuk-w10dirs",
    "Sodinokibi-largefiles", "Sodinokibi-w10dirs",
    "TeslaCrypt-largefiles", "TeslaCrypt-w10dirs",
    "WannaCry-largefiles", "WannaCry-w10dirs"
]  # Specify the folders to delete

delete_folders(source_directory, folders_list)


Deleted: Cerber-largefiles
Deleted: Cerber-w10dirs
Deleted: Darkside-largefiles
Deleted: Darkside-w10dirs
Deleted: GandCrab4-largefiles
Deleted: GandCrab4-w10dirs
Deleted: Ryuk-largefiles
Deleted: Ryuk-w10dirs
Deleted: Sodinokibi-largefiles
Deleted: Sodinokibi-w10dirs
Deleted: TeslaCrypt-largefiles
Deleted: TeslaCrypt-w10dirs
Deleted: WannaCry-largefiles
Deleted: WannaCry-w10dirs
Total folders deleted: 14


## Replace -0 in any row

In [34]:
for folder in folders:
    print("folder finished")
    for label in labels:
        os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
        dirs = sorted(os.listdir())
        dirs = np.array(dirs)
        
        # Shuffle directory
        for dir_idx in range(len(dirs)):
            print(dirs[dir_idx])
            os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
            files = sorted(os.listdir())

            # Read the files
            df_r = pd.read_csv(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[0]}', header=None)
            df_w = pd.read_csv(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[1]}', header=None)
            print(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}'+'/ata_read.csv')
            
            df_r = np.array(df_r)
            df_w = np.array(df_w)

            # Replacing -0 with 0
            df_r[df_r == -0] = 0
            df_w[df_w == -0] = 0

            # Save modified data back to the original CSV files
            pd.DataFrame(df_r).to_csv(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}'+'/ata_read.csv', header=False, index=False)
            pd.DataFrame(df_w).to_csv(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}'+'/ata_write.csv', header=False, index=False)

            print(f"Modified data saved back to files: {files[0]}, {files[1]}")
print("Processing completed!")


folder finished
AESCrypt-20200427_16-23-28
/home/HardDisk/Satang/thesis_proj/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-23-28/ata_read.csv
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-29-10
/home/HardDisk/Satang/thesis_proj/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-29-10/ata_read.csv
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-36-30
/home/HardDisk/Satang/thesis_proj/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-36-30/ata_read.csv
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-43-56
/home/HardDisk/Satang/thesis_proj/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-43-56/ata_read.csv
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-52-26
/home/HardDisk/Satang/thesis_proj/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-52-26/ata_read.csv
Modified data saved back to files: ata_read.csv, ata_write.csv

### Insert missing time

In [35]:
def find_missing_values_and_preview_then_save(file1, file2, auto_save=False):
    df1 = pd.read_csv(file1, header=None)
    df2 = pd.read_csv(file2, header=None)

    # Determine full range of seconds
    global_min = int(min(df1[0].min(), df2[0].min()))
    global_max = int(max(df1[0].max(), df2[0].max()))
    full_range = list(range(global_min, global_max + 1))

    def insert_missing(df, file_label):
        existing_seconds = set(df[0])
        missing_seconds = sorted(set(full_range) - existing_seconds)

        # print(f"\n[{file_label}] Missing seconds to insert: {missing_seconds}")

        for sec in missing_seconds:
            insert_idx = df[df[0] > sec].index.min()
            row = pd.Series([sec] + [np.nan] * (df.shape[1] - 1), index=df.columns)
            if pd.isna(insert_idx):
                df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
            else:
                top = df.iloc[:insert_idx]
                bottom = df.iloc[insert_idx:]
                df = pd.concat([top, pd.DataFrame([row]), bottom], ignore_index=True)

        # print(f"\nPreview of {file_label} after insertion:")
        # print(df.head(10))
        return df

    df1_fixed = insert_missing(df1, file1)
    df2_fixed = insert_missing(df2, file2)

    if not auto_save:
        user_input = input("\nSave changes to both files? (y/n): ").strip().lower()
    else:
        user_input = 'y'

    if user_input == 'y':
        df1_fixed.to_csv(file1, index=False, header=False)
        df2_fixed.to_csv(file2, index=False, header=False)
        print("✅ Files saved.")
    else:
        print("❌ Changes discarded (files not overwritten).")

    return df1_fixed, df2_fixed

In [36]:
# Processing loop
for folder in folders:
    print("Folder finished")
    for label in labels:
        os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
        dirs = sorted(os.listdir())
        dirs = np.array(dirs)
        
        # Shuffle directory
        for dir_idx in range(len(dirs)):
            print(dirs[dir_idx])
            os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
            files = sorted(os.listdir())

            # Read the files and process them
            file_r = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[0]}'
            file_w = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[1]}'
            
            find_missing_values_and_preview_then_save(file_r,file_w,auto_save=True)

            print(f"Modified data saved back to files: {files[0]}, {files[1]}")
print("Processing completed!")

Folder finished
AESCrypt-20200427_16-23-28
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-29-10
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-36-30
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-43-56
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-52-26
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-00-04
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-04-45
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-22-00
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-28-53
✅ Files saved.
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-40-13
✅ Files save

In [ ]:
# import os
# import numpy as np
# import pandas as pd

# for folder in folders:
#     print("folder finished")
#     for label in labels:
#         os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
#         dirs = sorted(os.listdir())
#         dirs = np.array(dirs)

#         # Shuffle directory
#         for dir_idx in range(len(dirs)):
#             print(dirs[dir_idx])
#             os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
#             files = sorted(os.listdir())

#             # Define file paths
#             file_read = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/ata_read.csv'
#             file_write = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/ata_write.csv'

#             # Process ata_read.csv if it exists
#             if os.path.exists(file_read):
#                 df_r = pd.read_csv(file_read, header=None)

#                 # Count unique data types per column
#                 type_counts_r = df_r.applymap(type).nunique()

#                 # Extract unique data types per column
#                 unique_types_r = df_r.applymap(type).apply(lambda col: col.unique())

#                 # Print results for ata_read.csv
#                 print(f"\nFile: {file_read}")
#                 print("Number of different data types per column:")
#                 print(type_counts_r.to_string())

#                 print("Unique data types per column:")
#                 for col_idx, types in unique_types_r.items():
#                     print(f"  Column {col_idx}: {', '.join(t.__name__ for t in types)}")
#             else:
#                 print(f"File not found: {file_read}")

#             # Process ata_write.csv if it exists
#             if os.path.exists(file_write):
#                 df_w = pd.read_csv(file_write, header=None)

#                 # Count unique data types per column
#                 type_counts_w = df_w.applymap(type).nunique()

#                 # Extract unique data types per column
#                 unique_types_w = df_w.applymap(type).apply(lambda col: col.unique())

#                 # Print results for ata_write.csv
#                 print(f"\nFile: {file_write}")
#                 print("Number of different data types per column:")
#                 print(type_counts_w.to_string())

#                 print("Unique data types per column:")
#                 for col_idx, types in unique_types_w.items():
#                     print(f"  Column {col_idx}: {', '.join(t.__name__ for t in types)}")
#             else:
#                 print(f"File not found: {file_write}")

# print("Check completed!")


### Convert float to int

In [ ]:
import os
import numpy as np
import pandas as pd

for folder in folders:
    print("folder finished")
    for label in labels:
        os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
        dirs = sorted(os.listdir())
        dirs = np.array(dirs)

        # Shuffle directory
        for dir_idx in range(len(dirs)):
            print(dirs[dir_idx])
            os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
            files = sorted(os.listdir())

            # Read the write file only
            file_path = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/ata_read.csv'
            df_w = pd.read_csv(file_path, header=None)

            # Convert DataFrame to NumPy
            np_w = df_w.to_numpy()

            # Convert only the first to fourth columns (0-3) to integers
            np_w[:, 0:4] = np_w[:, 0:4].astype(float).astype(int)

            # Convert back to DataFrame to preserve structure
            df_w = pd.DataFrame(np_w)

            # Debugging: Print data types after conversion
            print(f"\nAfter conversion - Data types in ata_write.csv ({files[1]}):\n", df_w.dtypes)

            # Save modified data back using NumPy to prevent Pandas auto-conversion
            np.savetxt(file_path, np_w, delimiter=",", fmt="%.10g")  # Keeps precision for non-int columns

            print(f"Converted float to int (columns 0-3) and saved file: {files[1]}")
        break

print("Processing completed!")


In [ ]:
import os
import numpy as np
import pandas as pd

for folder in folders:
    print("folder finished")
    for label in labels:
        os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
        dirs = sorted(os.listdir())
        dirs = np.array(dirs)

        # Shuffle directory
        for dir_idx in range(len(dirs)):
            print(dirs[dir_idx])
            os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
            files = sorted(os.listdir())

            # Read the write file only
            file_path = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/ata_write.csv'
            df_w = pd.read_csv(file_path, header=None)

            # Convert DataFrame to NumPy
            np_w = df_w.to_numpy()

            # Convert only the first to fourth columns (0-3) to integers
            np_w[:, 0:4] = np_w[:, 0:4].astype(float).astype(int)

            # Convert back to DataFrame to preserve structure
            df_w = pd.DataFrame(np_w)

            # Debugging: Print data types after conversion
            print(f"\nAfter conversion - Data types in ata_write.csv ({files[1]}):\n", df_w.dtypes)

            # Save modified data back using NumPy to prevent Pandas auto-conversion
            np.savetxt(file_path, np_w, delimiter=",", fmt="%.10g")  # Keeps precision for non-int columns

            print(f"Converted float to int (columns 0-3) and saved file: {files[1]}")
        break

print("Processing completed!")


### Check the types

In [ ]:
# import os
# import numpy as np
# import pandas as pd

# for folder in folders:
#     print("folder finished")
#     for label in labels:
#         os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
#         dirs = sorted(os.listdir())
#         dirs = np.array(dirs)

#         # Shuffle directory
#         for dir_idx in range(len(dirs)):
#             print(dirs[dir_idx])
#             os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
#             files = sorted(os.listdir())

#             # Define file paths
#             file_read = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/ata_read.csv'
#             file_write = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/ata_write.csv'

#             # Process ata_read.csv if it exists
#             if os.path.exists(file_read):
#                 df_r = pd.read_csv(file_read, header=None)

#                 # Count unique data types per column
#                 type_counts_r = df_r.applymap(type).nunique()

#                 # Extract unique data types per column
#                 unique_types_r = df_r.applymap(type).apply(lambda col: col.unique())

#                 # Print results for ata_read.csv
#                 print(f"\nFile: {file_read}")
#                 print("Number of different data types per column:")
#                 print(type_counts_r.to_string())

#                 print("Unique data types per column:")
#                 for col_idx, types in unique_types_r.items():
#                     print(f"  Column {col_idx}: {', '.join(t.__name__ for t in types)}")
#             else:
#                 print(f"File not found: {file_read}")

#             # Process ata_write.csv if it exists
#             if os.path.exists(file_write):
#                 df_w = pd.read_csv(file_write, header=None)

#                 # Count unique data types per column
#                 type_counts_w = df_w.applymap(type).nunique()

#                 # Extract unique data types per column
#                 unique_types_w = df_w.applymap(type).apply(lambda col: col.unique())

#                 # Print results for ata_write.csv
#                 print(f"\nFile: {file_write}")
#                 print("Number of different data types per column:")
#                 print(type_counts_w.to_string())

#                 print("Unique data types per column:")
#                 for col_idx, types in unique_types_w.items():
#                     print(f"  Column {col_idx}: {', '.join(t.__name__ for t in types)}")
#             else:
#                 print(f"File not found: {file_write}")

# print("Check completed!")


In [43]:
import numpy as np

corrupted_val = -9.223372037e+18
int_columns = [0, 1, 2, 3]  # Only these will be fixed

# Processing loop
for folder in folders:
    print("Folder finished")
    for label in labels:
        os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
        dirs = sorted(os.listdir())
        dirs = np.array(dirs)
        
        # Shuffle directory
        for dir_idx in range(len(dirs)):
            print(dirs[dir_idx])
            os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
            files = sorted(os.listdir())
            
            # Read the files and process them
            file_r = pd.read_csv(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[0]}',header=None)
            file_w = pd.read_csv(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[1]}',header=None)
            
            file_r_path = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[0]}'
            file_w_path = f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}/{files[1]}'
            # Step 3: Replace corrupted values and safely cast to 'Int64' per column
            for col in int_columns:
                file_r[col] = file_r[col].replace(corrupted_val, pd.NA)
                file_r[col] = file_r[col].astype("Int64")  # Nullable integer

                file_w[col] = file_w[col].replace(corrupted_val, pd.NA)
                file_w[col] = file_w[col].astype("Int64")  # Nullable integer

            # Save back
            file_r.to_csv(file_r_path, index=False, header=False)
            file_w.to_csv(file_w_path, index=False, header=False)

            print(f"Modified data saved back to files: {files[0]}, {files[1]}")
print("Processing completed!")

Folder finished
AESCrypt-20200427_16-23-28
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-29-10
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-36-30
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-43-56
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_16-52-26
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-00-04
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-04-45
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-22-00
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-28-53
Modified data saved back to files: ata_read.csv, ata_write.csv
AESCrypt-20200427_17-40-13
Modified data saved back to files: ata_read.csv, ata_write.csv
Cerber-20200805_23-19-47
Modified data saved back to files: ata_read.csv, ata_write.